# Practical set-up & Training data

This page will contains materials for the first tutorial session (April 19th).

The learning goals for the first tutorial are:

* preparing the Python requirements for practical exercises in the upcoming tutorials,
* test-running a few lines of code,
* familiarization with a few coding best practices,
* understanding key processing steps and terms of the first building block for training any language model -- the training data.

**Please try to complete the first block of this tutorial sheet (i.e., installation of requirements) AHEAD of the tutorial session**, ideally, while you have a stable internet connection. This way we can try to solve any problems that might have come up with the installation during the tutorial on Friday.

## Installing requirements

Throughout the semester, we will use Python, PyTorch and various packages for practical work. Both the in-tutorial exercise sheets and homework will require you to execute Python code yourself.
Please follow the steps below to set up the requirements (i.e., most packages required for completing exercises) that we will use in the course. We will most likely install more packages as we go during the semester, though.

You can do so either on your own machine, or by using [Google Colab](https://colab.research.google.com/). You can easily access the latter option by pressing the Colab icon at the top of the webook's page. Depending on your choice, please follow the respective requirement installation steps below.

### Colab

The advantage of using Colab is that you don't need to install software on your own machine; i.e., it is a safer option if you are not very comfortable with using Python on your own machine. Colab is a  platform provided by Google for free, and it also provides limited access to GPU computation (which will be useful fpor working with actual language models). Using it only requires a Google account.

For using a GPU on Colab, before executing your code, navigate to Runtime > Change runtime type > GPU > Save. Please note that the provided Colab computational resources are free, so please be mindful when using them. Further, Colab monitors GPU usage, so if it is used a lot very frequently, the user might not be able to access GPU run times for a while. 

Colab already provides Python as well as a number of basic packages. If you choose to use it, you will only need to install the more specific packages. Note that you will have to so *every time* you open a new Colab runtime. To test that you can access requirements for the class, please open this notebook in Colab (see above), uncomment and run the following line:

In [1]:
# !pip install datasets langchain torchrl llama-index bertviz wikipedia

### Local installation

Using your computer for local execution of all practical exercises might be a more advanced option. If you do so, we strongly encourage you to create an environment (e.g., with Conda) before installing any packages. Furthermore, ideally, check if you have a GPU suitable for deep learning because using a GPU will significantly speed up the work with language models. You can do so by checking your computer specs and finding out whether your GPU works with CUDA, MPS or ROCm. If you don't have a suitable GPU, you can use Colab for tasks that require GPU access. Finally, please note that we will download some pretrained models and some datasets which will occupy some of your local storage.

If you choose to use your own machine, please do the following steps:
* install Python >= 3.9
* create an environment (optional but recommended)
* download the requirements file [here](https://github.com/CogSciPrag/Understanding-LLMs-course/tree/main/understanding-llms/tutorials/files/requirements.txt)
* if you have a deep learning supporting GPU: 
  * please check [here](https://pytorch.org/get-started/locally/) which PyTorch version you need in order to use the GPU
  * please modify the first line of the requirements file to reflect the PyTorch version suitable for your machine (if needed)
  * please install the requirements from the requirements file (e.g., run: `pip install -r requirements.txt` once pip is available in your environment; adjust path to file if needed)
* if you do NOT have a deep learning supporting GPU:
  * please install the requirements from the requirements file (e.g., run: `pip install -r requirements.txt` once pip is available in your environment; adjust path to file if needed)

## Verifying requirement installation

Please run the following code cells to make sure that the key requirements were installed successfully. If you errors occur and you cannot solve them ahead of the tutorial, please don't be shy and let us know in the first tutorial!

In [2]:
# import packages

import torch
from transformers import AutoTokenizer
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
# check available computation device
# if you have a local GPU or if you are using a GPU on Colab, the following code should return "CUDA"
# if you are on Mac and have an > M1 chip, the following code should return "MPS"
# otherwise, it should return "CPU"

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Device: {device}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Device: {device}")
else:
    device = torch.device("cpu")
    print(f"Device: {device}")

Device: cpu


In [8]:
# test PyTorch

# randomly initialize a tensor of shape (5, 3)
x = torch.rand(5, 3).to(device)
print(x)
print("Shape of tensor x:", x.shape)
print("Device of tensor x:", x.device)

# initialize a tensor of shape (5, 3) with ones
y = torch.ones(5, 3).to(device)
print(y)

# multiply x and y
z = x * y
print(z)

tensor([[0.7460, 0.7034, 0.8912],
        [0.2328, 0.4722, 0.5892],
        [0.0167, 0.5969, 0.5472],
        [0.4784, 0.8627, 0.7594],
        [0.8009, 0.5393, 0.1019]])
Shape of tensor x: torch.Size([5, 3])
Device of tensor x: cpu
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.7460, 0.7034, 0.8912],
        [0.2328, 0.4722, 0.5892],
        [0.0167, 0.5969, 0.5472],
        [0.4784, 0.8627, 0.7594],
        [0.8009, 0.5393, 0.1019]])


In [9]:
# testing LangChain

# run a Wikipedia query, searching for the article "Attention is all you need"
# NB: requires an internet connection
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("Attention is all you need")

'Page: Attention Is All You Need\nSummary: "Attention Is All You Need" is a landmark 2017 research paper by Google. Authored by eight scientists, it was responsible for expanding 2014 attention mechanisms proposed by Bahdanau et. al. into a new deep learning architecture known as the transformer. The paper is considered by some to be a founding document for modern artificial intelligence, as transformers became the main architecture of large language models. At the time, the focus of the research was on improving Seq2seq techniques for machine translation, but even in their paper the authors saw the potential for other tasks like question answering and for what is now called multimodal Generative AI.\nThe paper\'s title is a reference to the song "All You Need Is Love" by the Beatles.As of 2024, the paper has been cited more than 100,000 times.\n\n\n\nPage: All You Need Is Kill\nSummary: All You Need Is Kill is a Japanese science fiction light novel by Hiroshi Sakurazaka with illustrat

In [10]:
# testing the package transformers which provides pre-trained language models
# and excellent infrastructure around them

# download (if not available yet) and load GPT-2 tokenizer
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
text = "Attention is all you need"
# tokenize the text (i.e., convert the string into a tensor of token IDs)
input_ids = tokenizer_gpt2(
    text,
    return_tensors="pt",
).to(device)

print("Input IDs:", input_ids)

Input IDs: {'input_ids': tensor([[8086, 1463,  318,  477,  345,  761]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


## Best practices for writing code

There is a lot of debate around best practices for writing, documenting and formatting Python code and their actual implementation in daily practice, and many people have different personal preferences. We are not committing to a particular side in this debate, but we do care about a few general aspects: 
* working with clean code 
* working with understandable code (i.e., commented, with understandable variable names etc)
* producing well-documented projects (e.g., supplied with relevant READMEs etc). Think: your work should be structured such that you could look at it in a year and be able to immediately what you did, how and why.

There are a few de facto standard *formatting* practices that help to keep Python code crisp and clean. Please take a look at these and adhere to these as much as you can (as so will we):
* [PEP8](https://pep8.org/): style guide for Python code defining e.g., variable naming conventions, how many spaces to use for indentation, how long single lines should be etc.
  * Here is an overview [video](https://www.youtube.com/watch?v=D4_s3q038I0) of some of the PEP8 conventions
  * There is handy software that reformats your code for you according to some of these conventions. Such software is often seamlessly integrated in IDEs. This includes for instance *Black* or *Ruff* Python formatters. They can be installed as extensions in, e.g., Visual Studio Code.
* doc-string writing
  * numpydoc style

In [ ]:
# example: bad formatting

def add(a,b):
    return a+b

# example: good formatting

def add(a, b):
    return a + b

# example: bad docstring

def add(a, b):
    return a + b

# example: good docstring

def add(a, b):
    """
    Add two numbers.

    Args:
        a (int): First number.
        b (int): Second number.

    Returns:
        int: Sum of a and b.
    """
    return a + b

There are also some hints regarding structuring larger projects and e.g. GitHub repositories (just fyi):
* 

These best practices will be useful to you beyond this class and possibly even beyond your studies when collaborating on other coding projects within teams or even by yourself. We do our best to stick to these guidelines ourselves and kindly urge you to do the same when submitting assignments and possibly projects.

## Understanding training data

Stay tuned!

* corpus:
* training data:
  * dataset:
  * test / validation data
* vocabulary
  * embedding (cf word2vec)
* batches:
* preprocessing:
* epochs:
* tokens:
  * see more details relevant to modern LLMs in a few sessions
* annotation
* EOS & concept of tight language models? (maybe next session)
  * re: smoothing